In [2]:
import pandas as pd
import numpy as np
import datetime
import pyodbc
isdw = pyodbc.connect('DSN=ISDW')
qdat = pyodbc.connect('DSN=QDAT')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sns
#import statsmodels.formula.api as sm


from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
daysback = 90
startdate = "'2020-06-28'"
#enddate = "'2020-05-31'"
enddate = "current date"
SQLQuery = """
WITH cte AS (
  SELECT LEFT(lot_Id,9) AS lot_id_base, family_Code, Last_test_date as date, yield
  FROM DMIW.PTileWaferFact pwf
  INNER JOIN DMIW_SYSTEMS.TestParm tp ON pwf.testParmKey = tp.testParmKey
  INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON pwf.testedWaferKey = tw.testedWaferKey
  WHERE Last_test_date >= (current date - {}) 
  AND Tech_id = '5H 14HP' AND Level = 'C5' AND tw.Calcdefs = 'DI511'
  AND weighted_Mean is not null AND abs(weighted_Mean) < 1e25 
  AND (ucase(tp.parm_Label) LIKE 'DI51_C5_M7125P125A_ST_A102_4M_PERFECT')
  ORDER BY lot_id_base, family_code, date
  )
  
SELECT MIN(date) AS date, lot_id_base, family_Code, AVG(yield) AS yield
FROM cte
WHERE date BETWEEN {} and {}
GROUP BY lot_id_base, family_Code
ORDER BY date DESC
""".format(daysback,startdate,enddate)

df = pd.read_sql(SQLQuery, isdw)

In [5]:
df.to_csv("~/Downloads/DGYields.csv")

In [4]:
df

,DATE,LOT_ID_BASE,FAMILY_CODE,YIELD
0,2020-09-05,8IQV33001,US,95.748235
1,2020-09-05,8IQV33002,US,94.604118
2,2020-09-05,8IQV33006,US,91.199444
3,2020-09-02,8IQV33005,US,93.957826
4,2020-08-22,8IQV27007,TM,91.172353
...,...,...,...,...
61,2020-06-29,8IQV22009,US,92.807059
62,2020-06-29,8IQV23004,9S,93.950000
63,2020-06-29,8IQV23005,9S,92.978421
64,2020-06-29,8IQV23007,TM,93.297059
